# Predictive Variables

In [ ]:
############### CHANGE THESE TO YOUR LIKING ###############

# specie = 'leptocybe-invasa' # 'leptocybe-invasa' # 'thaumastocoris-peregrinus' # 
region = region_train
# region = 'all' # 'east-asia' # 'south-east-asia' # 
training = True
#savefig = True
year00 = year0 # start year multiyear mean
year11 = year1 # end year multiyear mean

Future = Future
###########################################################

In [ ]:
import os

import numpy as np
import xarray as xr
import rioxarray

import pandas as pd
import geopandas as gpd
from geocube.api.core import make_geocube

import matplotlib.pyplot as plt
import matplotlib as mpl

import cartopy.io.shapereader as shapereader

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

import cdo
import shutil
import time
import sys
import glob
import subprocess
from shapely.geometry import Polygon
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling

In [ ]:
regions = {
    'east-asia': ['China', 'Taiwan', 'Japan', 'North Korea', 'South Korea'],
    'indo': ["Indonesia",'Malaysia','Singapore','Brunei','East Timor'],
    'sea': ['Myanmar', 'Cambodia', 'Laos', 'Philippines', 'Thailand', 'Vietnam'],
    'south-east-asia': ['Brunei', 'Myanmar', 'Cambodia', 'East Timor', 'Indonesia', 'Laos', 'Malaysia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],
    'australia': ['Australia'],
    'australasia' : ['Australia', 'New Zealand'],
    'india-sri-lanka' : ['Sri Lanka'],
    'all' : ['Australia','France','Italy','Portugal','South Africa','United States of America','Madagascar','Spain','Greece','Cyprus','Mexico','Kenya','Algeria','Israel','Egypt','Ethiopia','Ghana','Malawi','Mauritius','Morocco','Mozambique','Rwanda','Sierra Leone','United Republic of Tanzania','Tunisia','Uganda','Zimbabwe','China','India','Iran','Iraq','Jordan','Sri Lanka','Syria','Taiwan','Turkey','Malta','Montenegro','United Kingdom','Argentina','Brazil','Chile','Paraguay','Uruguay']
}

In [ ]:
# data_path = os.path.join(os.sep, 'scratch', 'aciar-fst', 'data')
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
docs_path = os.path.join(os.path.dirname(os.getcwd()), 'docs')
figs_path = os.path.join(os.path.dirname(os.getcwd()), 'figs')
out_path = os.path.join(os.path.dirname(os.getcwd()), 'out', specie)
input_path = os.path.join(out_path, 'input', 'train') if training else os.path.join(out_path, 'input', 'test')
dirtmp = os.path.join(input_path,'tmp') # temporary directory

In [ ]:
# Load shapefiles of countries
gdf_countries = gpd.read_file(os.path.join(input_path, '%s.shp' %region))
shapes = [shape for shape in gdf_countries.geometry]

minx, miny, maxx, maxy = gdf_countries.total_bounds
# bbox = [minx, maxx, miny, maxy]
bbox = [minx, miny, maxx, maxy]
print("Bounding box:", bbox)

minx, miny, maxx, maxy =  gdf_countries.total_bounds
poly = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny)])
gdf = gpd.GeoDataFrame(geometry=[poly], crs="EPSG:4326")

# fig, ax = plt.subplots(figsize=(6,6))
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# world.plot(ax=ax, linewidth=0.4, edgecolor="black", facecolor="lightgray")
# gdf.boundary.plot(ax=ax, linewidth=2, edgecolor="red")
# ax.set_xlim(minx-5, maxx+5)
# ax.set_ylim(miny-5, maxy+5)
# ax.set_title("Bounding box extent (EPSG:4326)")
# plt.tight_layout()
# plt.show()


# Load shapefiles of countries
# gdf_countries = gpd.read_file(os.path.join(input_path, '%s.shp' %region))
# shapes = [shape for shape in gdf_countries.geometry]

#shapes = gpd.read_file(os.path.join(input_path, '%s.shp' % region))

## 1. Bioclim variables (WorldClim)

In [ ]:
# 5. Define the full destination paths using the base input_path

hist_output_path = os.path.join(input_path, 'worldclim_train')
future_output_path = os.path.join(input_path, 'worldclim_train_future')

# 6. Rebuild the destination map for the script to use
destination_map = {
    'hist': hist_output_path,
    'future': future_output_path
}
bio_variables = range(1, 20)

if Future:
    print("\nFuture processing block is not yet implemented.")
else:
    print("\nStarting file copy, conversion, and CRS assignment process...")
    print("!!! WARNING: Files from previous models in the list will be overwritten. !!!")
    print("-" * 50)
    
    # Initialize counters
    file_copy_count = 0
    file_convert_count = 0
    file_missing_count = 0
    
    # Loop through each model in the list
    for model_prefix in models:
        print(f"\nProcessing Model: {model_prefix.strip('_')}")
    
        # Loop through each project type ('hist', 'future')
        for project_key, project_name in project_map.items():
            destination_folder = destination_map[project_key]
            print(f"  -> Project: {project_name} | Destination: {destination_folder}")
            os.makedirs(destination_folder, exist_ok=True)
    
                # Loop through each bioclimatic variable (1-19)
            for i in bio_variables:
                pattern = f"{model_prefix}_{project_name}_*_bio_{i}.tif"
                matches = glob.glob(os.path.join(bioclim_folder, pattern))
                
                if not matches:
                    print(f"bio_{i}: not found")
                    file_missing_count += 1
                    continue  # go to the next i
            
                # if matches are found → take the first one
                source_path = matches[0]
                print(f"bio_{i}: {source_path}")
            
                destination_filename = f"bio_{i}.tif"
                destination_path = os.path.join(destination_folder, destination_filename)
            
                # Check if the source file really exists (optional, since glob already checked)
                if not os.path.exists(source_path):
                    print(f"    - SKIPPED (Not Found): {source_path}")
                    file_missing_count += 1
                    continue
                
                try:
    
                # Check if this is a temperature variable (BIO1-11)
                    if 1 <= i <= 11:
                    # Open the raster to check its values
                        with rioxarray.open_rasterio(source_path) as src:
                        # Calculate the mean to detect the unit
                            mean_val = src.mean().item()
                                
                    # If the mean > threshold, assume Kelvin and convert
                        if mean_val > KELVIN_THRESHOLD:
                            celsius_data = src - 273.15
                            celsius_data.attrs['units'] = 'degrees Celsius'
                                    
                            # ADDED: Set the CRS before saving
                            celsius_data = celsius_data.rio.write_crs("EPSG:4326")

                                    
                            celsius_data.rio.to_raster(destination_path)
                            print(f"    - CONVERTED (K->C) & CRS Set: {destination_filename}")
                            file_convert_count += 1
                        else:
                            # MODIFIED: No longer a simple copy. Must open, set CRS, and save.
                            # The 'src' object is already open here.
                                
                            # ADDED: Set the CRS
                            src_with_crs = src.rio.write_crs("EPSG:4326")

                                    
                            src_with_crs.rio.to_raster(destination_path)
                            print(f"    - COPIED & CRS Set (Already Celsius): {destination_filename}")
                            file_copy_count += 1
                    else:
                        # MODIFIED: Logic for precipitation variables (BIO12-19).
                        # Must open, set CRS, and save instead of just copying.
                        with rioxarray.open_rasterio(source_path) as precip_src:
                                
                        # ADDED: Set the CRS
                            precip_src_with_crs = precip_src.rio.write_crs("EPSG:4326")


                            precip_src_with_crs.rio.to_raster(destination_path)
                            print(f"    - COPIED & CRS Set (Precipitation): {destination_filename}")
                            file_copy_count += 1
                except Exception as e:
                    print(f"    - ERROR processing {source_filename}: {e}")
    
    print("-" * 50)
    print("\nProcessing complete.")
    print(f"Summary: {file_convert_count} files converted, {file_copy_count} files copied with CRS set, {file_missing_count} files not found.")

In [ ]:
# import rasterio
# from rasterio.warp import reproject, Resampling
# from rasterio.enums import Resampling
# import numpy as np

# # Buka raster pertama (referensi)
# if Future:
#     with rasterio.open(os.path.join(input_path,'worldclim_train_future', "bio_1.tif")) as src1:
#     # with rasterio.open(os.path.join(input_path, "bio_1_all.tif")) as src1:
#         ref_profile = src1.profile
#         ref_transform = src1.transform
#         ref_bounds = src1.bounds
#         ref_crs = src1.crs
#         ref_width = src1.width
#         ref_height = src1.height
# else:
#     with rasterio.open(os.path.join(input_path,'worldclim_train', "bio_1.tif")) as src1:
#     # with rasterio.open(os.path.join(input_path, "bio_1_all.tif")) as src1:
#         ref_profile = src1.profile
#         ref_transform = src1.transform
#         ref_bounds = src1.bounds
#         ref_crs = src1.crs
#         ref_width = src1.width
#         ref_height = src1.height
        
# # Buka raster kedua dan sesuaikan dengan raster pertama
# with rasterio.open(os.path.join(input_path, "tmp", 'srtm_%s.tif' %region)) as src2:
#     data2 = src2.read(
#         out_shape=(src2.count, ref_height, ref_width),
#         resampling=Resampling.bilinear
#     )

#     # **Flip data secara eksplisit untuk memperbaiki orientasi**
#     data2 = np.flip(data2, axis=1)  # Flip pada sumbu Y
#     data2 = np.where(data2 < 0, np.nan, data2)
    
#     # Perbarui profil raster kedua agar sesuai dengan raster pertama
#     profile = src2.profile
#     profile.update({
#         "transform": ref_transform,
#         "width": ref_width,
#         "height": ref_height,
#         "crs": ref_crs,
#         "dtype": "float32"
#     })
    
#     # Simpan raster hasil yang sudah sesuai dengan raster pertama
#     with rasterio.open(os.path.join(input_path, 'srtm_%s.tif' %region), "w", **profile) as dst:
#         dst.write(data2.astype(np.float32))

In [ ]:

# Langkah 1: Buka raster referensi (src1) untuk mendapatkan propertinya
# Blok ini tetap sama untuk mendapatkan target georeferensi
if Future:
    ref_path = os.path.join(input_path, 'worldclim_train_future', "bio_1.tif")
else:
    ref_path = os.path.join(input_path, 'worldclim_train', "bio_1.tif")

with rasterio.open(ref_path) as src1:
    ref_profile = src1.profile
    ref_transform = src1.transform
    ref_crs = src1.crs
    ref_width = src1.width
    ref_height = src1.height
    ref_nodata = src1.nodata

# Langkah 2: Buka raster kedua (src2) dan gunakan reproject untuk memotong dan menyelaraskan
src2_path = os.path.join(input_path, "tmp", f'srtm_{region}.tif')
output_path = os.path.join(input_path, f'srtm_{region}.tif')

with rasterio.open(src2_path) as src2:
    # Buat array kosong sebagai tujuan dengan bentuk dan tipe data dari raster referensi
    destination = np.zeros((ref_height, ref_width), np.float32)

    # Lakukan reproject. Fungsi ini akan memotong dan menyesuaikan src2 ke grid src1
    reproject(
        source=rasterio.band(src2, 1),
        destination=destination,
        src_transform=src2.transform,
        src_crs=src2.crs,
        dst_transform=ref_transform,
        dst_crs=ref_crs,
        resampling=Resampling.bilinear
    )

    # Langkah 3: Lakukan manipulasi data tambahan seperti pada kode Anda
    # Catatan: Flip mungkin tidak lagi diperlukan karena reproject menangani orientasi,
    # tetapi tetap disertakan di sini jika data SRTM Anda memiliki orientasi terbalik.
    # Anda bisa mencoba menghapus baris flip ini dan lihat hasilnya.
    # destination = np.flip(destination, axis=0)  # Flip pada sumbu Y (axis 0 untuk array 2D)
    
    # Ganti nilai negatif dengan NaN (atau nilai nodata dari referensi)
    nodata_val = ref_nodata if ref_nodata is not None else -9999
    destination[destination < 0] = nodata_val
    
    # Perbarui profil untuk file output
    output_profile = ref_profile.copy()
    output_profile.update({
        'dtype': 'float32',
        'nodata': nodata_val
    })

    # Langkah 4: Simpan raster hasil yang sudah dipotong dan diselaraskan
    with rasterio.open(output_path, 'w', **output_profile) as dst:
        dst.write(destination, 1)

print(f"Raster telah berhasil dipotong, diselaraskan, dan disimpan di: {output_path}")

In [ ]:
# import os
# import xarray as xr
# import rioxarray as rio

# # Folder containing NetCDF files
# input_folder = os.path.join(dirnc)
# output_folder = os.path.join(dirtif)

# # List all NetCDF files
# nc_files = [f for f in os.listdir(input_folder) if f.endswith(".nc")]

# for nc_file in nc_files:
#     nc_path = os.path.join(input_folder, nc_file)
#     ds = xr.open_dataset(nc_path)  # Load NetCDF

#     # # Filter variables that contain "bioclim"
#     # print(f"Variabel dalam {nc_file}: {list(ds.variables.keys())}")
#     # print(f"Variabel dalam {nc_file}: {list(ds.data_vars.keys())}")
#     if 'time_bnds' in ds.variables:
#         ds = ds.drop_vars('time_bnds')
#         print(f"🗑️ Dropped 'time_bnds' from {nc_file}")


#     bioclim_vars = [var for var in ds.data_vars if "bioclim" in var.lower()]

#     if not bioclim_vars:
#         print(f"Skipping {nc_file}: No 'bioclim' variables found.")
#         continue

#     for variable_name in bioclim_vars:
#         da = ds[variable_name]
#         da = da.astype("float64")
#         da = da.where(np.isfinite(da), np.nan)  # Convert inf to NaN
#         # da = da.fillna(-9999.0)  # Replace NaN with -9999.0
     
        
#         # Rename dimensions if needed
#         if 'longitude' in da.dims and 'latitude' in da.dims:
#             da = da.rename({"longitude": "x", "latitude": "y"})
#         elif 'lon' in da.dims and 'lat' in da.dims:
#             da = da.rename({"lon": "x", "lat": "y"})

#         # Convert to GeoTIFF
#         tif_filename = f"{nc_file.replace('.nc', '')}.tif"
#         tif_path = os.path.join(output_folder, tif_filename)
#         da.rio.write_crs("EPSG:4326", inplace=True)  # Set CRS (modify if needed)
#         da.rio.to_raster(tif_path)

#         print(f"Converted {variable_name} from {nc_file} to {tif_path}")

# print("All applicable NetCDF files have been converted to GeoTIFF.")


In [ ]:
bioclim_names = {
    1: 'Annual Mean Temperature',
    2: 'Mean Diurnal Range (Mean of monthly (max temp - min temp))',
    3: 'Isothermality (BIO2/BIO7) (×100)',
    4: 'Temperature Seasonality (standard deviation ×100)',
    5: 'Max Temperature of Warmest Month',
    6: 'Min Temperature of Coldest Month',
    7: 'Temperature Annual Range (BIO5-BIO6)',
    8: 'Mean Temperature of Wettest Quarter',
    9: 'Mean Temperature of Driest Quarter',
    10: 'Mean Temperature of Warmest Quarter',
    11: 'Mean Temperature of Coldest Quarter',
    12: 'Annual Precipitation',
    13: 'Precipitation of Wettest Month',
    14: 'Precipitation of Driest Month',
    15: 'Precipitation Seasonality (Coefficient of Variation)',
    16: 'Precipitation of Wettest Quarter',
    17: 'Precipitation of Driest Quarter',
    18: 'Precipitation of Warmest Quarter',
    19: 'Precipitation of Coldest Quarter'
}



In [ ]:
bioclim_no = list(np.arange(1,20))#[1, 5, 6, 12, 13, 14]
file_names_worldclim = []
for no in bioclim_no:
    file_names_worldclim.append('bio_%s.tif' %no) 

In [ ]:
rasters, labels = [], []
for no, file_name in zip(bioclim_no, file_names_worldclim):
    if Future:
        wc_region = rioxarray.open_rasterio(os.path.join(input_path, 'worldclim_train_future', file_name), masked=True)
        wc_region = wc_region.rio.write_crs("EPSG:4326")
        wc_region = wc_region.rio.clip(geometries=shapes)
        wc_region = wc_region.fillna(-9999)  # replace NaN with -9999
        wc_region = wc_region.rio.write_nodata(-9999)
        wc_region.name = bioclim_names[no]
        wc_region.rio.to_raster(os.path.join(input_path, 'bio_%s_%s_future.tif' %(no, region))) #'wc2.1_10m_bio-%02d_%s.tif'
        
        rasters.append('bio_%s_%s_future.tif' %(no, region))
        labels.append('bioclim-%02d_future' %no)
    else:
        wc_region = rioxarray.open_rasterio(os.path.join(input_path, 'worldclim_train', file_name), masked=True)
        wc_region = wc_region.rio.write_crs("EPSG:4326")
        wc_region = wc_region.rio.clip(geometries=shapes)
        wc_region = wc_region.fillna(-9999)  # replace NaN with -9999
        wc_region = wc_region.rio.write_nodata(-9999)
        wc_region.name = bioclim_names[no]
        wc_region.rio.to_raster(os.path.join(input_path, 'bio_%s_%s.tif' %(no, region))) #'wc2.1_10m_bio-%02d_%s.tif'
        
        rasters.append('bio_%s_%s.tif' %(no, region))
        labels.append('bioclim-%02d' %no)


titles = ['bio-%02d: %s' %(no, name) for (no, name) in bioclim_names.items()]

# rasters, labels, titles

In [1]:
# bioclim_plot = input('Provide bioclim numbers to plot (max of 9):').split()
# bioclim_plot = [int(bc) for bc in bioclim_plot]
bioclim_plot = [1, 5, 6, 8, 16, 17]
#bioclim_plot = [i for i in range(1, 20)]

In [ ]:
if len(bioclim_plot) <= 3:
    nrows, ncols = 1, len(bioclim_plot)
    figsize = (18, 4)
elif 3 < len(bioclim_plot) <= 6:
    nrows, ncols = 2, 3
    figsize = (18, 7)
elif 6 < len(bioclim_plot) <= 9:
    nrows, ncols = 3, 3
    figsize = (18, 10)


fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, sharex=True, sharey=True)
for iax, bc_no in enumerate(bioclim_plot):
    r, c = iax // ncols, iax % ncols
    axes = ax[c] if nrows == 1 else ax[r, c]
    ds = rioxarray.open_rasterio(os.path.join(input_path, rasters[bc_no - 1]), masked=True)
    if bc_no < 12:
        cmap = plt.cm.Spectral_r
    elif bc_no == 12:
        cmap = plt.cm.BuGn
    else:
        cmap = plt.cm.BrBG
    
    gdf_countries.plot(ax=axes, facecolor='lightgray', edgecolor='k')
    #pcol = ds.plot(ax=ax[r, c], cmap=attrs[titles[iax]][0], vmin=attrs[titles[iax]][1], vmax=attrs[titles[iax]][2], add_colorbar=False, add_labels=False)
    #pcol = ds.plot(ax=axes, cmap=cmap, add_colorbar=False, add_labels=False)
    
    # Extract the first layer of the raster for plotting
    # raster_data = ds.isel(band=0)
    raster_data = ds.isel()
    
    # Plot the raster data
    pcol = raster_data.plot(
        ax=axes,
        cmap=cmap,
        add_colorbar=False,
        add_labels=False,
    )
    axes.set_title(titles[bc_no - 1])
    
    plt.colorbar(pcol, ax=axes, extend='both')

plt.tight_layout()

In [ ]:
if savefig:
    if Future:
        fig.savefig(os.path.join(figs_path, '03a_wordclim_bioclim_%s_%s_future.png' %(region,models)), transparent=True)
    else:
        fig.savefig(os.path.join(figs_path, '03a_wordclim_bioclim_%s_%s.png' %(region,models)), transparent=True)